In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch==1.4.0 torchvision==0.5.0 ninja torch-encoding

     |████████████████████████████████| 753.4MB 21kB/s 
     |████████████████████████████████| 4.0MB 44.0MB/s 
     |████████████████████████████████| 112kB 56.4MB/s 
     |████████████████████████████████| 133kB 57.2MB/s 
     |████████████████████████████████| 163kB 57.2MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [3]:
%cd /content/
!git clone https://github.com/zhanghang1989/PyTorch-Encoding
%cd /content/PyTorch-Encoding/
!python setup.py clean
!python setup.py install
!pip install git+https://github.com/zhanghang1989/PyTorch-Encoding/
!pip install git+https://github.com/facebookresearch/fvcore.git

/content
Cloning into 'PyTorch-Encoding'...
remote: Enumerating objects: 1801, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 1801 (delta 19), reused 42 (delta 19), pack-reused 1752
Receiving objects: 100% (1801/1801), 21.20 MiB | 36.67 MiB/s, done.
Resolving deltas: 100% (986/986), done.
/content/PyTorch-Encoding
-- Building version 1.2.2b20210418
c++:  ['/content/PyTorch-Encoding/encoding/lib/cpu/encoding_cpu.cpp', '/content/PyTorch-Encoding/encoding/lib/cpu/rectify_cpu.cpp', '/content/PyTorch-Encoding/encoding/lib/cpu/syncbn_cpu.cpp', '/content/PyTorch-Encoding/encoding/lib/cpu/roi_align_cpu.cpp', '/content/PyTorch-Encoding/encoding/lib/cpu/nms_cpu.cpp', '/content/PyTorch-Encoding/encoding/lib/cpu/operator.cpp']
cuda:  ['/content/PyTorch-Encoding/encoding/lib/gpu/operator.cpp', '/content/PyTorch-Encoding/encoding/lib/gpu/nms_kernel.cu', '/content/PyTorch-Encoding/encoding/lib/gpu/encoding_kernel.cu', '/content/PyTor

In [11]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [12]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-00962a48-a68a-ba48-34af-c434e8d6de64)


In [13]:
!python /content/PyTorch-Encoding/experiments/segmentation/train_dist.py --dataset citys --resume "/content/runs/citys/deeplab/resnest50/default/model_best.pth.tar" --model deeplab --aux --backbone resnest50 --crop-size 248 --batch-size 8 --epochs 110

Namespace(aux=True, aux_weight=0.2, backbone='resnest50', base_size=520, batch_size=8, checkname='default', crop_size=248, dataset='citys', dist_backend='nccl', dist_url='tcp://localhost:23456', epochs=110, eval=False, export=None, ft=False, lr=0.005, lr_scheduler='poly', model='deeplab', model_zoo=None, momentum=0.9, rank=0, rectify=False, rectify_avg=False, resume='/content/runs/citys/deeplab/resnest50/default/model_best.pth.tar', se_loss=False, se_weight=0.2, seed=1, start_epoch=0, test_batch_size=16, test_folder=None, test_val=False, train_split='train', weight_decay=0.0001, workers=8, world_size=1)
rank: 0 / 1
BaseDataset: base_size 520, crop_size 248
Found 2975 images in the folder /content/drive/MyDrive/data/citys/leftImg8bit/train
Found 500 images in the folder /content/drive/MyDrive/data/citys/leftImg8bit/val
DeepLabV3(
  (pretrained): ResNet(
    (conv1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): DistSyncBat

In [ ]:
import warnings
try:
    from queue import Queue
except ImportError:
    from Queue import Queue

import torch
from torch.nn.modules.batchnorm import _BatchNorm

from encoding.utils.misc import EncodingDeprecationWarning
from encoding.functions import *

class DistSyncBatchNorm(_BatchNorm):
    r"""Cross-GPU Synchronized Batch normalization (SyncBN)

    Standard BN [1]_ implementation only normalize the data within each device (GPU).
    SyncBN normalizes the input within the whole mini-batch.
    We follow the sync-onece implmentation described in the paper [2]_ .
    Please see the design idea in the `notes <./notes/syncbn.html>`_.

    .. math::

        y = \frac{x - mean[x]}{ \sqrt{Var[x] + \epsilon}} * gamma + beta

    The mean and standard-deviation are calculated per-channel over
    the mini-batches and gamma and beta are learnable parameter vectors
    of size C (where C is the input size).

    During training, this layer keeps a running estimate of its computed mean
    and variance. The running sum is kept with a default momentum of 0.1.

    During evaluation, this running mean/variance is used for normalization.

    Because the BatchNorm is done over the `C` dimension, computing statistics
    on `(N, H, W)` slices, it's common terminology to call this Spatial BatchNorm

    Args:
        num_features: num_features from an expected input of
            size batch_size x num_features x height x width
        eps: a value added to the denominator for numerical stability.
            Default: 1e-5
        momentum: the value used for the running_mean and running_var
            computation. Default: 0.1
        sync: a boolean value that when set to ``True``, synchronize across
            different gpus. Default: ``True``
        activation : str
            Name of the activation functions, one of: `leaky_relu` or `none`.
        slope : float
            Negative slope for the `leaky_relu` activation.

    Shape:
        - Input: :math:`(N, C, H, W)`
        - Output: :math:`(N, C, H, W)` (same shape as input)

    Reference:
        .. [1] Ioffe, Sergey, and Christian Szegedy. "Batch normalization: Accelerating deep network training by reducing internal covariate shift." *ICML 2015*
        .. [2] Hang Zhang, Kristin Dana, Jianping Shi, Zhongyue Zhang, Xiaogang Wang, Ambrish Tyagi, and Amit Agrawal. "Context Encoding for Semantic Segmentation." *CVPR 2018*

    Examples:
        >>> m = DistSyncBatchNorm(100)
        >>> net = torch.nn.parallel.DistributedDataParallel(m)
        >>> output = net(input)
    """
    def __init__(self, num_features, eps=1e-5, momentum=0.1, process_group=None):
        super(DistSyncBatchNorm, self).__init__(num_features, eps=eps, momentum=momentum, affine=True, track_running_stats=True)
        self.process_group = process_group

    def forward(self, x):
        need_sync = self.training or not self.track_running_stats
        process_group = None
        if need_sync:
            process_group = torch.distributed.group.WORLD
            if self.process_group:
                process_group = self.process_group
            world_size = torch.distributed.get_world_size(process_group)
            need_sync = world_size > 1

        # Resize the input to (B, C, -1).
        input_shape = x.size()
        x = x.view(input_shape[0], self.num_features, -1)
        #def forward(ctx, x, gamma, beta, running_mean, running_var, eps, momentum, training, process_group):
        y = dist_syncbatchnorm(x, self.weight, self.bias, self.running_mean, self.running_var,
                               self.eps, self.momentum, self.training, process_group)

        #_var = _exs - _ex ** 2
        #running_mean.mul_((1 - ctx.momentum)).add_(ctx.momentum * _ex)
        #running_var.mul_((1 - ctx.momentum)).add_(ctx.momentum * _var)
        return y.view(input_shape)


In [ ]:
import encoding
model = encoding.models.get_segmentation_model('deeplab', dataset='citys',
                                   backbone="resnest50", aux='True',
                                   se_loss='False', norm_layer=DistSyncBatchNorm,
                                   base_size=520, crop_size=248)

In [ ]:
import torch
torch.cuda.empty_cache()
# Get the model
with torch.no_grad():
    checkpoint = torch.load('/content/runs/citys/deeplab/resnest50/default/model_best.pth.tar')
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    model.cuda()

# Prepare the image
filename = '/content/zurich_000050_000019_leftImg8bit.png'
img = encoding.utils.load_image(filename).cuda().unsqueeze(0)

# Make prediction
output = model.evaluate(img.cuda())
predict = torch.max(output, 1)[1].cpu().numpy() + 1

# Get color pallete for visualization
mask = encoding.utils.get_mask_pallete(predict, 'citys')
mask.save('/content/output70.png')

In [ ]:
%pwd

In [ ]:
%ls